# 🧠 Brain MRI Preprocessing Pipeline (U-Net Ready)

This pipeline prepares the LGG Brain MRI dataset for training a U-Net model. It performs patient-wise splitting, image resizing, normalization, mask binarization, and saves the processed data in `.npy` format.

---

## ✅ Steps Performed

1. **Set Paths**:
   - `Data_directory = '/Users/aman/Python/BRAIN MRI U-Net/archive/lgg-mri-segmentation/kaggle_3m'`
   - `OUTPUT_DIR = os.path.abspath("processed_data")`

2. **Create Output Folders**:
   - For each split (`train`, `val`, `test`), create `images/` and `masks/` subdirectories under `processed_data`.

3. **List and Sort Patients**:
   - Extract folders starting with `TCGA` (i.e., valid patient data).

4. **Split Patients**:
   - 70% for training, 15% for validation, and 15% for testing using `train_test_split`.

5. **Define Image Normalization**:
   - Normalize each image using Min-Max scaling: `(img - min) / (max - min)`.

6. **Preprocess Each Image-Mask Pair**:
   - Load `.tif` image and corresponding `_mask.tif`.
   - Resize both to `128×128` (`BILINEAR` for images, `NEAREST` for masks).
   - Normalize image to `[0.0, 1.0]`.
   - Binarize mask using threshold `mask >= 128 → 1 else 0`.
   - Log any empty masks (i.e., masks with no positive pixels).
   - Save image and mask as `.npy` files with names like:
     - `TCGA_XXXX_YYYY_1.npy`
     - `TCGA_XXXX_YYYY_1_mask.npy`

7. **Execution**:
   - The script prints logs for missing masks and empty masks.
   - Final confirmation message prints the save directory path.

---

## 💾 Final Directory Structure

processed_data/
├── train/
│   ├── images/
│   └── masks/
├── val/
│   ├── images/
│   └── masks/
└── test/
    ├── images/
    └── masks/

Each `.npy` file has shape `(128, 128)`:
- Image: dtype `float32`, values in `[0.0, 1.0]`
- Mask: dtype `uint8`, values in `{0, 1}`

---

## 📝 Example Output Log

🔍 Processing train set (76 patients)  
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_1  
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_10  
...  
✅ Success! Preprocessed data saved at:  
/Users/aman/Python/BRAIN MRI U-Net/processed_data

---

## 🧠 Notes

- Empty masks may be filtered out during training.
- Missing masks are skipped automatically.
- Data is now ready for loading into PyTorch/TensorFlow for model training.

> This pipeline ensures consistent preprocessing for brain tumor segmentation using U-Net models.


In [1]:
import os
from PIL import Image



In [2]:
# --- Step 1: Configuration Parameters ---

# Path to the raw dataset directory
DATA_DIR = '/Users/aman/Python/BRAIN MRI U-Net/archive/lgg-mri-segmentation/kaggle_3m'

# Path where processed data will be saved
OUTPUT_DIR = os.path.abspath("processed_data")

# Target image and mask size
IMAGE_SIZE = (128, 128)

# Resampling methods for resizing
RESAMPLE_METHOD_IMAGE = Image.Resampling.BILINEAR
RESAMPLE_METHOD_MASK = Image.Resampling.NEAREST


In [3]:
# --- Step 2: Dataset Validation & Directory Setup ---

# Check if the dataset directory exists
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Dataset not found at: {DATA_DIR}")

# Create output directories for each data split
for split in ["train", "val", "test"]:
    for folder in ["images", "masks"]:
        os.makedirs(os.path.join(OUTPUT_DIR, split, folder), exist_ok=True)


In [4]:
# --- Step 3: Patient Listing and Sorting ---

# List all patient folders that start with 'TCGA' and are directories
patient_folders = [
    f for f in os.listdir(DATA_DIR)
    if f.startswith("TCGA") and os.path.isdir(os.path.join(DATA_DIR, f))
]

# Sort the patient folders for consistent ordering
patient_folders.sort()



In [5]:
from sklearn.model_selection import train_test_split

# --- Step 4: Train/Val/Test Split (Patient-wise) ---

# Split patient folders into train+val and test sets (15% for testing)
train_val, test = train_test_split(patient_folders, test_size=0.15, random_state=42)

# Further split train_val into train and val sets (~15% of total for validation)
train, val = train_test_split(train_val, test_size=0.176, random_state=42)

# Store the splits in a global dictionary
splits = {
    "train": train,
    "val": val,
    "test": test
}


In [6]:
import numpy as np

# --- Step 5: Image Normalization Function ---

def normalize(img_array):
    """
    Normalize image pixel values to the range [0, 1].

    Parameters:
        img_array (np.ndarray): Input grayscale image array.

    Returns:
        np.ndarray: Normalized image.
    """
    img_array = img_array.astype(np.float32)
    img_min, img_max = img_array.min(), img_array.max()
    
    if img_max > img_min:
        return (img_array - img_min) / (img_max - img_min)
    else:
        return np.zeros_like(img_array)
        



In [7]:
from PIL import Image
from tqdm import tqdm

# --- Step 6: Preprocessing and Saving Function ---

def preprocess_and_save(patient_list, split_name):
    """
    Process and save images and masks for a given data split.

    Parameters:
        patient_list (list): List of patient folder names for the split.
        split_name (str): One of 'train', 'val', or 'test'.
    """
    print(f"\n🔍 Processing {split_name} set ({len(patient_list)} patients)")

    for patient in tqdm(patient_list, desc=f"{split_name.capitalize()} Patients"):
        patient_path = os.path.join(DATA_DIR, patient)

        # Get all image files (exclude mask files)
        images = sorted([
            f for f in os.listdir(patient_path)
            if f.endswith(".tif") and not f.endswith("_mask.tif")
        ])

        for img_file in images:
            base_name = os.path.splitext(img_file)[0]
            mask_file = f"{base_name}_mask.tif"
            mask_path = os.path.join(patient_path, mask_file)

            # Skip if corresponding mask does not exist
            if not os.path.exists(mask_path):
                print(f"❌ Mask missing for {img_file} in {patient}, skipping.")
                continue

            try:
                # Load and preprocess image and mask
                img = Image.open(os.path.join(patient_path, img_file)).convert('L')
                mask = Image.open(mask_path).convert('L')

                img = img.resize(IMAGE_SIZE, RESAMPLE_METHOD_IMAGE)
                mask = mask.resize(IMAGE_SIZE, RESAMPLE_METHOD_MASK)

                img_np = normalize(np.array(img))  # normalize image
                mask_np = np.array(mask)
                mask_np = (mask_np >= 128).astype(np.uint8)  # binarize mask


                # Optional warning for empty masks
                if not np.any(mask_np):
                    print(f"⚠️ Empty mask for {patient}_{base_name}")

                # Save the processed data
                save_base = f"{patient}_{base_name}"
                np.save(os.path.join(OUTPUT_DIR, split_name, "images", save_base + ".npy"), img_np)
                np.save(os.path.join(OUTPUT_DIR, split_name, "masks", save_base + "_mask.npy"), mask_np)

            except Exception as e:
                print(f"❗ Error processing {img_file} for {patient}: {str(e)}")
                continue




In [8]:
# --- Step 7: Main Execution Block ---

if __name__ == "__main__":
    for split_name, patients in splits.items():
        preprocess_and_save(patients, split_name)

    print("\n✅ Success! Preprocessed data saved at:")
    print(f"{OUTPUT_DIR}")




🔍 Processing train set (76 patients)


Train Patients:   3%|▎         | 2/76 [00:00<00:04, 17.47it/s]

⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_1
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_10
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_11
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_12
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_13
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_14
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_15
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_16
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_2
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_29
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_3
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_30
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_31
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_32
⚠️ Empty mask for TCGA_DU_7013_19860523_TCGA_DU_7013_19860523_33
⚠️ Empty mask for TCGA_DU_70

Train Patients:   7%|▋         | 5/76 [00:00<00:03, 22.36it/s]

⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_22
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_23
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_24
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_3
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_4
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_5
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_6
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_7
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_8
⚠️ Empty mask for TCGA_CS_6188_20010812_TCGA_CS_6188_20010812_9
⚠️ Empty mask for TCGA_HT_8113_19930809_TCGA_HT_8113_19930809_1
⚠️ Empty mask for TCGA_HT_8113_19930809_TCGA_HT_8113_19930809_10
⚠️ Empty mask for TCGA_HT_8113_19930809_TCGA_HT_8113_19930809_11
⚠️ Empty mask for TCGA_HT_8113_19930809_TCGA_HT_8113_19930809_2
⚠️ Empty mask for TCGA_HT_8113_19930809_TCGA_HT_8113_19930809_21
⚠️ Empty mask for TCGA_HT_8113_199

...

⚠️ Empty mask for TCGA_HT_7605_19950916_TCGA_HT_7605_19950916_9
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_1
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_10
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_2
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_25
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_26
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_27
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_28
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_3
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_4
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_5
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_6
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_7
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_8
⚠️ Empty mask for TCGA_HT_7608_19940304_TCGA_HT_7608_19940304_9
⚠️ Empty mask for TCGA_CS_4941_1996

Test Patients:  88%|████████▊ | 15/17 [00:00<00:00, 26.42it/s]

⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_2
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_20
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_21
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_22
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_23
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_24
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_25
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_26
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_27
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_28
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_29
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_3
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_30
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_31
⚠️ Empty mask for TCGA_DU_5852_19950709_TCGA_DU_5852_19950709_32
⚠️ Empty mask for TCGA_DU_5

Test Patients: 100%|██████████| 17/17 [00:00<00:00, 25.71it/s]

⚠️ Empty mask for TCGA_FG_7637_20000922_TCGA_FG_7637_20000922_51
⚠️ Empty mask for TCGA_FG_7637_20000922_TCGA_FG_7637_20000922_6
⚠️ Empty mask for TCGA_FG_7637_20000922_TCGA_FG_7637_20000922_7
⚠️ Empty mask for TCGA_FG_7637_20000922_TCGA_FG_7637_20000922_8
⚠️ Empty mask for TCGA_FG_7637_20000922_TCGA_FG_7637_20000922_9
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_1
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_2
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_23
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_24
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_25
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_26
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_27
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_28
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_29
⚠️ Empty mask for TCGA_DU_A5TS_19970726_TCGA_DU_A5TS_19970726_3
⚠️ Empty mask for TCGA_DU_A5TS_1